In [5]:
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET,QUICK_SMART_MODEL,ANTHROPIC_API_KEY,basic_config_agent
from dynamic_graph.outputsModels import *
from dynamic_graph.main_graph_functions import run_collaborative_planning
from prompts import PromptEngine

In [6]:
# --- Konfiguracja czatu grupowego ---
main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}
router_agent_configuration = {"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=QUICK_SMART_MODEL, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}

In [4]:
if __name__ == '__main__':
    # Konfiguracja dla lekkiego i szybkiego modelu Rutera
    
    
    # Wysokopoziomowa misja od użytkownika
    mission = """
    Zbuduj prosty, sekwencyjny przepływ pracy do weryfikacji modelu przyczynowego.
    Załóż, że każdy krok (węzeł) kończy się natychmiast (działa synchronicznie).
    Plan powinien składać się z kroków: load_data, discover_causality, a na końcu validate_model.
    """
    
    # Uruchom całą fazę planowania jedną komendą
    final_plan = run_collaborative_planning(mission, router_agent_configuration)
    
    if final_plan:
        print("\n" + "="*50)
        print("### OTRZYMANY PLAN GOTOWY DO WYKONANIA ###")
        print(final_plan.model_dump_json(indent=2, by_alias=True))
        
        # W kolejnym kroku, ten obiekt `final_plan` zostałby przekazany
        # do "Fabryki Grafów", aby zbudować i uruchomić przepływ.
        # build_and_run_graph(final_plan, ...)
    else:
        print("\nNie udało się wypracować planu. Sprawdź logi w `reports/`.")

    
    


### ROZPOCZYNANIE FAZY PROJEKTOWEJ ###
--- RUTER AGENTÓW: Uruchamiam uniwersalny wybór zespołu... ---
INFO [StrucRes]: Próba 1/3...

>>>>>>>> USING AUTO REPLY...
INFO [StrucRes]: Odpowiedź poprawna i zwalidowana.
--- RUTER AGENTÓW: Wybrany zespół -> {'thought_process': "Misja wymaga zbudowania sekwencyjnego przepływu pracy dla weryfikacji modelu przyczynowego, obejmującego ładowanie danych, odkrywanie przyczynowości i walidację modelu. 'Ekspert_Przyczynowosci' jest idealny do etapu 'discover_causality' oraz ogólnego planowania modelu przyczynowego. 'Analityk_Danych' jest niezbędny do etapu 'load_data' (czyszczenie i przygotowanie danych) oraz do aspektów walidacji danych w 'validate_model'. Obaj są kluczowi do pokrycia wszystkich kroków planu. 'Krytyk_Jakosci' jest jedynym dostępnym ekspertem, którego rola to 'konstruktywna krytyka planów', co czyni go idealnym wyborem na krytyka.", 'planners': ['Ekspert_Przyczynowosci', 'Analityk_Danych'], 'critic': 'Krytyk_Jakosci'} ---

--- URUCHAM